In [1]:
import pickle
import random
import re
import time

from collections import Counter
from heapq import nlargest
from lxml import etree
from sys import getsizeof

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer



NAMESPACE = '{http://www.mediawiki.org/xml/export-0.10/}'
ARTICLE_ID = '0'
NAMESPACE_TAG = NAMESPACE + 'ns'
PAGE_TAG = NAMESPACE + 'page'
TITLE_TAG = NAMESPACE + 'title'
REDIRECT_TAG = NAMESPACE + 'redirect'
TEXT_TAG = NAMESPACE + 'text'
SOURCE = './wiki-dump/enwiki-1gb'
DEST = './wiki-dump/'


-----
Utils:
-----

In [ ]:
# Iteration over xml with low ram consumption
def fast_iter(context, func, dest):
    for event, elem in context:
        # Execute operations over xml node
        stop = func(elem, dest)
        
        # Clear data read
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
        
        if stop:
            break
    
    del context


--------------
Split wikipedia dump xml:
---------------

In [ ]:
start_time = time.time()

context = etree.iterparse('./wiki-dump/enwiki-1gb', tag=PAGE_TAG , events=('end',))

xml_header = '<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="en">\n'
xml_tail = '</mediawiki>\n'

n = 2
name = './wiki-dump/enwiki-part'

for i in range(1, n + 1):
    filename = name + str(i)
    with open(filename, 'a') as f:
        f.write(xml_header)

# Splits wikipedia dump into n parts choosing randomly the articles of each part
for event, elem in context:
    # Execute operations over xml node
    temp_prob = 0
    rand = random()
    for i in range(1, n + 1):
        if temp_prob <= rand < i / n:
            filename = name + str(i)
            with open(filename, 'ab') as f:
                wiki_page = etree.tostring(elem)
                f.write(wiki_page)
            break
        else:
            temp_prob = i / n
    
    # Clear data read
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context

for i in range(1, n + 1):
    filename = name + str(i)
    with open(filename, 'a') as f:
        f.write(xml_tail)

print("--- %s seconds ---" % (time.time() - start_time))

--------------
Extract keyphrases:
-------------------

In [ ]:
# clean_pattern = '\[\[(image:|file:|category:)(.+)\]\]'
# clean_pattern2 = '\[\[(image:|file:|category:)(.*(<br>\n)*)*]]'
extract_pattern = '\[\[([^][]+)\]\]'

ignore = ['Image:', 'File:', 'Category:', 'Q:', 'Wikipedia:', 'image:', 'file:', 'category:', 'q:', 'wikipedia:']
          #, '<span style=', 'talk:', 'Talk:', '<font style=']

# clean_regex = re.compile(clean_pattern, re.IGNORECASE)
extract_regex = re.compile(extract_pattern, re.IGNORECASE)

context = etree.iterparse('./wiki-dump/mini/enwiki-test1gb', tag=PAGE_TAG , events=('end',))

start_time = time.time()

vocabulary = Counter()
for event, elem in context:
    iterator = elem.iterchildren(tag=NAMESPACE_TAG)
    namespace_id = iterator.__next__().text
    
    # Extract data only over articles, exclude: talks, mediawiki, portal, user, etc
    if namespace_id == ARTICLE_ID:
        new_keys = []
        # Extract titles of pages
        try:
            iterator = elem.iterchildren(tag=REDIRECT_TAG)
            redirect_child = iterator.__next__()
            page_title = redirect_child.attrib['title'].lower()

        except StopIteration:
            iterator = elem.iterchildren(tag=TITLE_TAG)
            title_child = iterator.__next__()
            page_title = title_child.text.lower()

        new_keys.append(page_title)

        # Extract keyphrases in other morphological form
        iterator = elem.iterdescendants(tag=TEXT_TAG)
        text = iterator.__next__().text
        keyphrases = extract_regex.findall(text)
        for key in keyphrases:
            if not any(x in key for x in ignore) and len(key) > 0:
                new_keys.append(key.split('|')[-1].lower())

        vocabulary.update(new_keys)

    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context

# Delete keyphrases that occurs less than 5 times
for key, value in list(vocabulary.items()):
    if value <= 5:
        del vocabulary[key]


print('Vocabulary: {} ngrams --- {} MB'.format(len(vocabulary), getsizeof(vocabulary) / 1024**2))
print("--- %s seconds ---" % (time.time() - start_time))


# Save vocabulary
with open('./wiki-dump/vocabulary_dict', 'wb') as f:
    pickle.dump(vocabulary, f)

# Vocabulary: 204163 ngrams - 97.37 sec - enwiki-test1gb

In [ ]:
top_10 = nlargest(100, vocabulary.keys(), key=lambda x: len(x.split()))
# print(top_100)
print([len(i.split()) for i in top_10])
# print(sum([len(i.split()) for i in vocabulary.keys()]) / len(vocabulary))  # ==> 2.09


------------------
Count N-grams in articles:
--------------------------

In [ ]:
start_time = time.time()

with open('./wiki-dump/vocabulary_dict', 'rb') as f:
    vocabulary = pickle.load(f)

vectorizer = CountVectorizer(lowercase=True, ngram_range=(1, 16), vocabulary=vocabulary.keys(), token_pattern='[\S]+')

context = etree.iterparse('./wiki-dump/mini/enwiki-test1gb', tag=PAGE_TAG , events=('end',))
articles = []
keys_count = np.zeros((1, len(vocabulary.keys())))

for i, (event, elem) in enumerate(context):
    iterator = elem.iterchildren(tag=NAMESPACE_TAG)
    namespace_id = iterator.__next__().text

    if namespace_id == ARTICLE_ID:
        if i % 10000 == 0:
            m = vectorizer.transform(articles)
            m = m.sum(axis=0)
            keys_count = np.add(keys_count, m)
            articles = []
        else:
            iterator = elem.iterdescendants(tag=TEXT_TAG)
            text = iterator.__next__().text
            articles.append(text)

    # Clear data read
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context


# Leave this or not?
# I take advantage of this loop to build a dict from vocabulary keys to their index in the array
key_to_index = dict()
for index, keyword in enumerate(vectorizer.get_feature_names()):
    keys_count[0, index] += vocabulary[keyword]
    key_to_index[keyword] = index


print("--- %s seconds ---" % (time.time() - start_time))

# Save data
# with open('wiki-dump/vectorizer', 'wb') as f:
#     pickle.dump(vectorizer, f)

with open('wiki-dump/keys_count', 'wb') as f:
    pickle.dump(keys_count, f)
    
with open('wiki-dump/key_to_index', 'wb') as f:
    pickle.dump(key_to_index, f)

# 114.47 sec, ngram=(1, 20) 1,000
# 86.39 sec, ngram=(1, 16) 1,000
# 47.87 sec, ngram=(1, 10) 1,000
# 406.82 sec, ngram=(1, 16) 5,000
# 843.35 sec, ngram=(1, 16) 10,000
# 3443.72 sec, ngram=(1, 16) iterando de a 10,000
# 4310.21 sec, ngram=(1, 21) iterando de a 10,000

In [ ]:
print(keys_count[0, 52156])
print(vectorizer.get_feature_names()[2])
key_to_index.items()

In [ ]:
print(ft[-2])
print(vectorizer.vocabulary_['caught in the act'])
print(keys_count[0, 1561442])

-----------
Ranking method, Keyphraseness:
------------------------------

In [ ]:
keyphraseness = np.zeros((1, len(vocabulary.keys())))

for index, keyword in enumerate(vectorizer.get_feature_names()):
    keyphraseness[0, index] = vocabulary[keyword] / keys_count[0, index]

# Save data
with open('wiki-dump/keyphraseness', 'wb') as f:
    pickle.dump(keyphraseness, f)

------------------------
Evaluation of Keyphraseness:
----------------------------

In [3]:
with open('wiki-dump/vocabulary_dict', 'rb') as f:
    vocabulary = pickle.load(f)
    
with open('wiki-dump/keyphraseness', 'rb') as f:
    keyphraseness = pickle.load(f)

context = etree.iterparse('./wiki-dump/mini/enwiki-train', tag=PAGE_TAG , events=('end',))

count = 1000
articles = []
for event, elem in context:
    iterator = elem.iterchildren(tag=NAMESPACE_TAG)
    namespace_id = iterator.__next__().text
    
    if len(articles) < count:
        # Extract data only over articles, exclude: talks, mediawiki, portal, user, etc
        if namespace_id == ARTICLE_ID:
            try:
                iterator = elem.iterchildren(tag=REDIRECT_TAG)
                redirect_child = iterator.__next__()

            except StopIteration:
                iterator = elem.iterdescendants(tag=TEXT_TAG)
                text = iterator.__next__().text.lower()
                articles.append(text)
    else:
        break

    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context


In [ ]:
extract_pattern = '\[\[([^][]+)\]\]'
extract_regex = re.compile(extract_pattern, re.IGNORECASE)
ignore = ['Image:', 'File:', 'Category:', 'Q:', 'image:', 'file:', 'category:', 'q:']

for article in articles:
    # Gold keyphrases extracted for wikipedia articles
    gold_keyphrases = set([key.split('|')[-1].lower() for key in extract_regex.findall(article)
                           if not any(x in key for x in ignore) and len(key) > 0])
    
    # Predicted keyphrases
    vectorizer = CountVectorizer(lowercase=True, ngram_range=(1, 16), 
                                 vocabulary=vocabulary.keys(), token_pattern='[\S]+')
    m = vectorizer.transform([article])
    
    
    

In [103]:
article = articles[0]
vectorizer = CountVectorizer(lowercase=True, ngram_range=(1, 16), vocabulary=vocabulary.keys())
m = vectorizer.transform([article])

In [104]:
rows, cols = m.nonzero()

In [105]:
zipped = list(zip(cols, keyphraseness[0, cols]))

In [106]:
percent_6 = int(len(article.split(' ')) * .06)
percent_6

112

In [ ]:
top = nlargest(percent_6, zipped, key=lambda t: t[1])
# top

In [108]:
extract_pattern = '\[\[([^][]+)\]\]'
extract_regex = re.compile(extract_pattern, re.IGNORECASE)
ignore = ['Image:', 'File:', 'Category:', 'Q:', 'image:', 'file:', 'category:', 'q:']

gold_keyphrases = set([key.split('|')[-1].lower() for key in extract_regex.findall(article)
                       if not any(x in key for x in ignore) and len(key) > 0])

In [109]:
features_names = vectorizer.get_feature_names()
predicted_keyphrases = [features_names[i] for i, prob in top]

In [110]:
gold_keyphrases.intersection(set(predicted_keyphrases))

{'ernest hemingway',
 'gene kelly',
 'leonard bernstein',
 'new york',
 'new york philharmonic',
 'pablo picasso',
 'piccolo',
 'snare drum',
 'soprano saxophone',
 'triangle',
 'university of michigan'}

In [102]:
features_names[140033]

'classical'

In [112]:
predicted_keyphrases

['instrument',
 'conjunction',
 'she',
 'play',
 'poem',
 'others',
 'kennedy',
 'leonard bernstein',
 'ira gershwin',
 'victor',
 'soprano saxophone',
 'loose',
 'de',
 'modern',
 'four',
 'angeles',
 'he',
 'lc',
 'los angeles',
 'calendar',
 'tone poem',
 'chaplin',
 'gene kelly',
 'horns',
 'picasso',
 'artist',
 'new york philharmonic',
 'radio',
 'new york',
 'first episode',
 'rca',
 'the united states',
 'composer',
 'all',
 'war',
 'net',
 'walking',
 'slow',
 'pablo picasso',
 'states',
 'impressions',
 'caron',
 'place',
 'document',
 'to',
 'web',
 'working',
 'master',
 'compiled',
 'ami',
 'california',
 'bowl',
 'pound',
 'snare drum',
 'orchestra',
 'ernest',
 'hand',
 'when',
 'william',
 'gershwin',
 'back',
 'the street',
 'last',
 'wood',
 'thumb',
 'multiple',
 'complete',
 'prior',
 '22',
 'theories',
 'fame',
 'audience',
 'portrait',
 'premiere',
 '30',
 'one',
 'recorded',
 'triangle',
 'made',
 'plus',
 'http',
 'episode',
 'the sounds',
 'style',
 'memorial',

In [117]:
vocabulary['22']

54